<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Bi-directional-problem" data-toc-modified-id="Bi-directional-problem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bi-directional problem</a></span><ul class="toc-item"><li><span><a href="#Run" data-toc-modified-id="Run-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Run</a></span><ul class="toc-item"><li><span><a href="#Init-Graph" data-toc-modified-id="Init-Graph-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Init Graph</a></span></li><li><span><a href="#Run-ICU" data-toc-modified-id="Run-ICU-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Run ICU</a></span></li><li><span><a href="#Run-Outer-Cost-Update" data-toc-modified-id="Run-Outer-Cost-Update-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Run Outer Cost Update</a></span></li></ul></li></ul></li></ul></div>

# Description

This file aims at implementing both versions of the algorithm (with rebalancers update inside and outside of the loop) and at comparing their performance and convergence to a solution. 

Comparison is done on a simple graph. The dummy problem we talked about. 

# Load

In [12]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from routines_icu import *
from helpers_icu import *
import cvxpy as cp
from FW_icu import *

from FW_OuterUpdate import solve

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Bi-directional problem

In [13]:
#parameters
alpha=0.15
beta=4
POTENTIAL_SHIFT=90
INVERSE_DEMAND_SHIFT=80

ZC_EDGE=1

In [14]:
L_dum=10
#to adapt
#to be read from file if necessary
nodes_list=['1','2','2_p','R','1_p']
edge_list=[('1','2'),('2','1'),('2','2_p'),('1','R'),('1','2_p'),('1','1_p'),('2','1_p'),('2','R')]
k_list=[10,10,3,10,ZC_EDGE,2,ZC_EDGE,10] 
l_list=[10,10,1,L_dum,0,1,0,L_dum]
t_list=[10,10,10,10,ZC_EDGE,10,ZC_EDGE,10]
phi_list=[phi(l_list[i],t_list[i]) for i in range(len(l_list))]
is_negative=[0,0,1,0,0,1,0,0]

nodes_pots=[('2_p',POTENTIAL_SHIFT),('1_p',POTENTIAL_SHIFT)]

In [24]:
dummy_nodes=dict()
dummy_nodes['2_p']='2'
dummy_nodes['1_p']='1'

## Run

### Init Graph

In [144]:
#Note: it is the same to write OD[(a,b)] as OD[a,b]

OD=dict()
N=10
OD['1','2_p']=N
# OD['2','1_p']=N

In [145]:
G=nx.DiGraph()
G.add_nodes_from(nodes_list)
G.add_edges_from(edge_list)

In [146]:
G=initEdgeAttr(G,edge_list,k_list,phi_list,is_negative)
G=initNodeAttr(G,nodes_pots)
G=update_costs(G,INVERSE_DEMAND_SHIFT)

#Create a feasible solution for the passengers
G=init_flows(G,OD) 

G=update_costs(G,INVERSE_DEMAND_SHIFT)

In [107]:
G_0=G.copy()

### Run ICU

First algorithm with the updates in the loop

In [108]:
G_k,y_list,opt_res,OD_list=modified_FW(G_0.copy(),OD.copy(),edge_list,dummy_nodes,maxIter=400,
                         step='fixed',rebalancer_smoothing=True, ri_smoothing=False,evolving_bounds=False)

In [110]:
OD_list[-1]

{('1', '2_p'): 10, ('1', 'R'): 0, ('2', 'R'): 8.607645060332542}

In [109]:
print_final_flows(G_k)

('1', '2')  :  8.621499252181273
('1', 'R')  :  8.583242447140236
('1', '2_p')  :  1.3785007478187314
('1', '1_p')  :  0.0
('2', '1')  :  8.583242447140236
('2', '2_p')  :  8.621499252181273
('2', '1_p')  :  0.0
('2', 'R')  :  10.0


Why does 2-R have a non-zero value? 

### Run Outer Cost Update

Second algorithm with the updates inside the loop

In [153]:
OD

{('1', '2_p'): 10}

Ca decrease mais pas de maniere monotone... 

In [155]:
G_,ri_ = solve(G_0.copy(),OD.copy(),edge_list,dummy_nodes,tol=10**-6)

Below is the initialized flows
('1', '2')  :  10
('1', 'R')  :  0
('1', '2_p')  :  0
('1', '1_p')  :  0
('2', '1')  :  0
('2', '2_p')  :  10
('2', '1_p')  :  0
('2', 'R')  :  0
CURRENT RI_k
{'1': 10, '2': -10, '2_p': 0, 'R': 0, '1_p': 0}
CURRENT OD: {('1', '2_p'): 10, ('1', 'R'): 0, ('2', 'R'): 10}
('1', '2')  :  36.0
('1', 'R')  :  41.4
('1', '2_p')  :  90.0
('1', '1_p')  :  13.599999999999994
('2', '1')  :  41.4
('2', '2_p')  :  13.599999999999994
('2', '1_p')  :  90.0
('2', 'R')  :  10000000000000000000000
('1', '2')  :  0
('1', 'R')  :  10
('1', '2_p')  :  10
('1', '1_p')  :  0
('2', '1')  :  10
('2', '2_p')  :  0
('2', '1_p')  :  0
('2', 'R')  :  0
134.6666666666667
20.403845393938468
187.71318735287636
39.261427848533266
12.204905916133214
155.73774696121254
58.53146103842528
13.724291935710752
19.581048240824543
108.77489440244307
52.260252608731335
20.217770721153656
3.671828958992073
32.93689520911283
70.92734187534676
38.9906001784267
18.508390111404978
5.92948006005723
10.07

1.532441893292713
1.2180784045804245
0.9108344576699636
0.6105759822595473
0.31717117080114576
0.03049044859378114
1.6171136126033048
1.6079921303418274
1.296620607711091
0.9921889572981257
0.6945685113705196
0.40363273915178866
0.11925721909726406
1.0248456351693562
1.677835792495344
1.3694875737077297
1.0679049812020764
0.7729644805176008
0.48454455682905007
0.20252568921451086
0.4714353196262664
1.7424362558097641
1.4371317476459495
1.138423956214254
0.8461942257726207
0.5603258107157529
0.2807038516619116
0.0072153515507693555
1.6867860329664381
1.512649403151329
1.216586860995065
0.9268470802499138
0.6433178032239653
0.36588855817737453
0.09445063709145529
1.1055517771890777
1.582765487763126
1.2893819700038023
1.0021703041420693
0.7210225076101722
0.44583228997944957
0.1764950322607035
0.5611002287594147
1.6478975980789698
1.3572171801331194
1.072562039593251
0.7938282672162131
0.5209135580646205
0.25371719221236333
0.050501093071943615
1.708428128940227
1.4204664335918693
1.1383

0.5907824319193651
0.7700888644538253
0.6309547586123472
0.49330523883060096
0.3571272029470407
0.22240765259509487
0.08913369255496661
0.27472660079456546
0.8147825356308829
0.676157550640113
0.5390003753360304
0.40329815114377254
0.26903812057612697
0.1362076266070602
0.00479411204558744
0.8077637302380367
0.7227778175638804
0.5860932902275948
0.45084758649589673
0.3170281830347932
0.1846226543764118
0.05361867231313511
0.4894220843341941
0.7678816825975758
0.6316783862146167
0.4968979725682914
0.3635281472859333
0.23155671134675515
0.10097156049705802
0.1815665299170064
0.8115243976559441
0.675810177700285
0.5415030875103923
0.4085910565997892
0.27706210739690107
0.14690435468122587
0.01810600501977433
0.704998844510385
0.7213056321120348
0.5874533077735578
0.4549808868490999
0.323876607157942
0.19412879653214077
0.0657258722722247
0.39486081260508854
0.7653522170973588
0.6319626482346052
0.4999379992671322
0.36926671872184613
0.2399373428735032
0.11193849521757215
0.094735192341033

0.02897007567582932
0.3964871331350537
0.5258589512333316
0.43207625980629416
0.33898248573096623
0.2465734507681958
0.15484499943954688
0.06379299892934398
0.17093092158881973
0.5589190460035098
0.46535174592287376
0.37246811648799394
0.2802640324324983
0.18873539084333402
0.09787811106527045
0.007688134605189134
0.5271538377601814
0.499245703720419
0.40656636075027563
0.31456145555776516
0.22322693679333838
0.1325587749695103
0.042552962367253144
0.3010592406405408
0.5323921748132108
0.43992033783797524
0.3481178692882594
0.2569807686009824
0.16650505668734183
0.07668677584214834
0.08018491330716415
0.5648097431609074
0.472548453881231
0.38095150224179974
0.2900149376999366
0.19973483080828203
0.11010727312624624
0.021128377130409604
0.4326805974226886
0.5057812771231482
0.4143841937722499
0.3236425983198643
0.23355261002652128
0.14411036879150174
0.05531203506437521
0.2112200339379342
0.5382914993418666
0.44709732885637266
0.35655378345511224
0.26665703038192134
0.17740325715820404


0.02245249051185283
0.28544196357967166
0.387958422831602
0.319386068758007
0.2511865327962397
0.18335814320214183
0.11589923496714505
0.04880814979678405
0.11515625793862228
0.41337010915226813
0.34490777211327384
0.27681617798087643
0.20909367060986028
0.1417386005020802
0.07474932478487517
0.008124207188913601
0.374187056956373
0.37075189702909483
0.3027660000456365
0.23514715522620122
0.1678937283651436
0.10100409179496239
0.0344766243649417
0.20377865955816787
0.396177674494794
0.3282988778600924
0.260785109628884
0.19363475071855873
0.1268461884988028
0.0604178167712206
0.03631160217736351
0.4211927613083901
0.35342241275395736
0.2860150797650647
0.21896915821460539
0.1522830503439252
0.08595516474134968
0.01998391632393748
0.29357012049433706
0.37886366345122724
0.311560575816344
0.2446169257841575
0.17803113026006656
0.11180161241465214
0.045926801664407435
0.12594913307566458
0.40389732195119166
0.3366998229237623
0.2698597981316624
0.20337567898440057
0.13724590307498374
0.07

...

0.03364190358877295
0.06676375264319745
0.05551839206853515
0.04428335218074153
0.03305862518766389
0.021844203302350707
0.010640078743108461
0.0035570425727655675
0.07141849564695235
0.06017574366559586
0.04894330297034344
0.037721165781135824
0.02650932432285913
0.015307770826169076
0.004116497526510443
0.04538877947607034
0.06484120478414468
0.05361135395206418
0.04239179725529141
0.031182526931431198
0.019983535222991122
0.008794814377679927
0.015312242192862868
0.06949415390789682
0.05826690081796926
0.04704993250115308
0.035843241206436005
0.024646819188475888
0.013460658706961226
0.002284752026980641
0.05706255181138431
0.06293052591325932
0.05171613669622843
0.04051201516922731
0.029318153598822505
0.018134544256938057
0.0069611794203296995
0.026994494030077476
0.0675816604836541
0.056369858230226555
0.045168314342333815
0.033977021098479554
0.022795970782723884
0.011625155683688604
0.00046456809559458634
0.06866375828866467
0.061031626903499614
0.04983265142399773
0.0386439172

0.0647314219505688
0.05418504412097036
0.04364774544326622
0.03311951948782621
0.022600359829311856
0.012090260045553691
0.0015892137192139444
0.05720316066600617
0.05857219435938532
0.04803716739733943
0.03751120541883779
0.026994302007366855
0.016486450750562653
0.005987645240196571
0.028923181061031755
0.06294830562595166
0.05241555692079203
0.04189186546599899
0.03137722485459449
0.02087162868323844
0.01037507055325193
0.0007223476393960482
0.0673134094806187
0.05678294553587193
0.04626153111409792
0.03574915981761251
0.025245825252852683
0.014751521030262893
0.0042662407644797895
0.03989765222348751
0.061157442882816326
0.05063829783345852
0.04012818820602604
0.02962710761595133
0.019135049683228544
0.008652008031631908
0.01170424003350945
0.06552095053028495
0.05500408134457757
0.044496239883173416
0.03399741977131043
0.023507614637438223
0.013026818114980188
0.002555023840855597
0.05080818078062066
0.0593769479163484
0.048871366998923804
0.038374799756525135
0.027887239827592225

0.05708978890766048
0.04687828998234522
0.03667530896404969
0.026480840008332507
0.01629487727397816
0.006117414923707543
0.0260281760433827
0.061333301778347504
0.05112394214786775
0.040923093381351805
0.030730749642444266
0.020546905098214552
0.010371553919313214
0.0002046902802845807
0.06395785214283478
0.055376257279476704
0.0451775338050453
0.034987308335928446
0.024805575047381012
0.01463232811819637
0.004467561731374303
0.036547846983296495
0.059618213615479476
0.04942162138463857
0.039233520141777944
0.029053904070565295
0.018882767358149977
0.008720104195305112
0.009212133577875647
0.06384983982690073
0.053655384757675506
0.04346941366458168
0.03329192073906029
0.02312290017667351
0.01296234617664993
0.0028102529407476595
0.04711675348318067
0.057895801846854056
0.04771195396232031
0.037536577254286385
0.027369665926457287
0.017211214185351764
0.007061216242064461
0.019788168551770013
0.062125904960453227
0.051944186192287134
0.041770931614451
0.03160613543876423
0.02144979188

0.034345888757172816
0.024466592325211423
0.014595286015136821
0.004731964512146192
0.032915045595530046
0.0582455439316476
0.04835225678648669
0.038466963992192404
0.028589660234672465
0.018720340203344676
0.008858998590483225
0.006387411712182711
0.062352886703536115
0.05246161045543829
0.042578322152436954
0.032703016488
0.02283568815866488
0.012976331864052781
0.003124942307067613
0.04316513908423758
0.05657722673032595
0.04669593658043425
0.036822622681683015
0.02695727973815565
0.017099902456578775
0.0072504855472885765
0.016644242030793066
0.0606831298748034
0.05080384327780507
0.040932526549710246
0.03106917440160828
0.021213781547231747
0.011366342704107524
0.0015268525926188659
0.053359147888158986
0.05491799137087128
0.045048665504097585
0.035187297853156
0.02533388313892715
0.015488416086384404
0.005650891422931181
0.026845092069606835
0.059022441096573175
0.049155111482917846
0.03929573372515449
0.029444302551453916
0.019600812693667535
0.009765258886744732
0.0004005881226

KeyboardInterrupt: 

In [135]:
G_[-1].get_edge_data('1','1_p')

{'k': 2,
 'phi': 3.6,
 'sign': -1,
 'f_m': 0.0,
 'f_r': 0.0,
 'cost': 13.599999999999994}

In [130]:
G.get_edge_data('1','2')

{'k': 10, 'phi': 36.0, 'sign': 1, 'f_m': 10, 'f_r': 0, 'cost': 41.4}

In [119]:
ri_

[{'1': 4.652786721467814e-09,
  '2': -4.652786721467814e-09,
  '2_p': 0,
  'R': 0.0,
  '1_p': 0},
 {'1': 3.634804239771751e-18,
  '2': -3.634804239771751e-18,
  '2_p': 0,
  'R': 0.0,
  '1_p': 0}]

In [126]:
print_final_flows(G_)

('1', '2')  :  0.0
('1', 'R')  :  0.0
('1', '2_p')  :  10.0
('1', '1_p')  :  0.0
('2', '1')  :  0.0
('2', '2_p')  :  0.0
('2', '1_p')  :  0.0
('2', 'R')  :  0.0


In [127]:
print_final_cost(G_)

('1', '2')  :  41.4
('1', 'R')  :  36.0
('1', '2_p')  :  90.0
('1', '1_p')  :  13.599999999999994
('2', '1')  :  36.0
('2', '2_p')  :  80.26666666666668
('2', '1_p')  :  90.0
('2', 'R')  :  36.0


In [103]:
OD

{('1', '2_p'): 10, ('2', '1_p'): 10}